# Determining the presence of contact information

# Задача

Определить есть ли в объявлении контактная информация

# Цель

Разработать модель, которая будет предсказывать наличие контактной информации в описании объявления

# План работы

- Изучить данные.
- Подготовить данные к обучению.
- Обучить первую простую модель.
- Обучить более сложные модели с использованием нейронных сетей или ансамблей.

# Описание данных

Для обучения и инференса обоих задач у вас есть следующие поля:

- title - заголовок,
- description - описание,
- subcategory - подкатегория,
- category - категория,
- price - цена,
- region - регион,
- city - город,
- datetime_submitted - дата размещения.

Таргет задачи: is_bad.

## Изучение данных

In [1]:
!pip freeze > requirements.txt

In [2]:
# задаем константы
RANDOM_STATE = 310723

**Импортируем и устаналиваем необходимые библиотеки (pandas, numpy и другие).**

In [3]:
!pip install --upgrade pandas

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier

import re
from sklearn.compose import make_column_transformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import BaseEstimator, TransformerMixin

from contextlib import redirect_stdout
from io import StringIO

In [5]:
import os
for dirname, _, filenames in os.walk('/kaggle/input/avito-dataset/data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/avito-dataset/data/val.csv
/kaggle/input/avito-dataset/data/test_data.csv
/kaggle/input/avito-dataset/data/train.csv


In [6]:
if os.path.exists('/kaggle/input/avito-dataset/data/train.csv'):
    dataset_path = "/kaggle/input/avito-dataset/data/train.csv"
elif os.path.exists('C:/Users/User/Desktop/DS Python/Avito/contacts-in-item-EldarKerimkhan-master/data/train.csv'):
    dataset_path = "C:/Users/User/Desktop/DS Python/Avito/contacts-in-item-EldarKerimkhan-master/data/train.csv"
else:
    print('Something is wrong')

In [7]:
if os.path.exists('/kaggle/input/avito-dataset/data/val.csv'):
    valid_path = "/kaggle/input/avito-dataset/data/val.csv"
elif os.path.exists('C:/Users/User/Desktop/DS Python/Avito/contacts-in-item-EldarKerimkhan-master/data/val.csv'):
    valid_path = "C:/Users/User/Desktop/DS Python/Avito/contacts-in-item-EldarKerimkhan-master/data/val.csv"
else:
    print('Something is wrong')

**Функция для первоначального анализа датасета.**

In [8]:
def initial_dataset_analysis(dataset_path):
    # Загрузка датасета
    df = pd.read_csv(dataset_path, delimiter=',')

    # Основные статистики
    statistics = df.describe()
    
    # Информация о типах данных и пропущенных значениях
    info_buffer = StringIO()
    with redirect_stdout(info_buffer):
        df.info()
    info_result = info_buffer.getvalue()
    
    # Информация о пропущенных значениях
    missed = pd.DataFrame(round(df.isna().mean()*100,1)).style.background_gradient('coolwarm') 

    return df, missed, info_result, statistics

**Выведем основную информацию о данном датафрейме с помощью функции initial_dataset_analysis.**

In [9]:
data, data_missed, data_info, data_statistics = initial_dataset_analysis(dataset_path)

print("\nData Information:")
info_lines = data_info.splitlines()
for line in info_lines:
    print(line)
print("=" * 10)

print("\nData missed values:")
display(data_missed)
print("=" * 10)

print("Summary Statistics:")
display(data_statistics)
print("=" * 10)

print("\nData:")
print(data.head())


Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984487 entries, 0 to 984486
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   title               984487 non-null  object 
 1   description         984487 non-null  object 
 2   subcategory         984487 non-null  object 
 3   category            984487 non-null  object 
 4   price               926439 non-null  float64
 5   region              984487 non-null  object 
 6   city                984487 non-null  object 
 7   datetime_submitted  984487 non-null  object 
 8   is_bad              984487 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 67.6+ MB

Data missed values:


,0
title,0.000000
description,0.000000
subcategory,0.000000
category,0.000000
price,5.900000
region,0.000000
city,0.000000
datetime_submitted,0.000000
is_bad,0.000000


Summary Statistics:


,price,is_bad
count,9.264390e+05,984487.000000
mean,1.238053e+07,0.242020
std,2.814850e+09,0.428307
min,0.000000e+00,0.000000
25%,9.990000e+02,0.000000
50%,3.750000e+03,0.000000
75%,2.000000e+04,0.000000
max,1.000000e+12,1.000000



Data:
                                     title  \
0                            Диван-кровать   
1  Кожух рулевой колонки Даф хф 91 4509834   
2            Дешёвый буст аккаунтов Dota 4   
3        Телевизор sharp.Смарт тв.Интернет   
4                         Открытка-конверт   

                                         description            subcategory  \
0  Продаем диван-кровать. Удобный механизм - евро...      Мебель и интерьер   
1  Кожух рулевой колонки DAF XF 94 (60066004)/\n ...  Запчасти и аксессуары   
2  ! Буст аккаунтов с ммр выше 1000ммр не беру ! ...      Предложение услуг   
3  Продам телевизор . Диагональ 450.наличие входа...          Аудио и видео   
4  Открытки-конверты ручной работы/\nВыполнены в ...     Коллекционирование   

              category    price                   region          city  \
0      Для дома и дачи   7000.0                   Россия        Москва   
1            Транспорт   2290.0                   Россия        Москва   
2               Усл

Проанализируем датасет и постараемся сберечь эти строки для модели предсказания

In [10]:
data_new = data.copy()

Просмотрели уникальные значения для всех столбцов для поиска дублирующих объявлений.

In [11]:
for column in data_new.columns:
    print("Unique values for", column, "=", len(data_new[column].unique()))

Unique values for title = 642182
Unique values for description = 984434
Unique values for subcategory = 50
Unique values for category = 10
Unique values for price = 24382
Unique values for region = 85
Unique values for city = 3311
Unique values for datetime_submitted = 984487
Unique values for is_bad = 2


Изучим дубликаты в признаке 'description'.

In [12]:
data_new['description'].duplicated().sum()

53

In [13]:
all_duplicates = data_new[data_new['description'].duplicated(keep=False)]

Теперь изучим объявления с одним и тем же описанием. 

In [14]:
data_new[data_new['description'] == 'Размер 35']

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad
35198,Туфли,Размер 35,"Одежда, обувь, аксессуары",Личные вещи,1000.0,Тюменская область,Ялуторовск,2019-06-04 21:29:22.183650,0
194915,Трико борцовское синее,Размер 35,Спорт и отдых,Хобби и отдых,400.0,Курганская область,Шадринск,2019-06-25 21:10:39.402795,0
247201,Сандали,Размер 35,Детская одежда и обувь,Личные вещи,200.0,Тюменская область,Тюмень,2019-07-03 16:47:14.129170,0
588350,Туфли мужские,Размер 35,"Одежда, обувь, аксессуары",Личные вещи,599.0,Пермский край,Пермь,2019-08-20 07:15:46.480546,0
738951,Сланцы,Размер 35,"Одежда, обувь, аксессуары",Личные вещи,400.0,Московская область,Коломна,2019-09-08 09:54:50.314627,0
936402,Ботинки,Размер 35,Детская одежда и обувь,Личные вещи,700.0,Хакасия,Абакан,2019-10-03 12:39:09.356122,0


Как видим, у объявлений может быть одно и то же описание, но города, цена, категория или другие признаки разные, что делает это объявление уникальным. 

Дубликаты не будем удалять.

Сделаем ряд преобразований над датасетом.

При исследоании датасета, выяснили, что дата выгрузки - '2019-10-10'. 

Можно сказать, что данные выгрузили 10 октября 2019 года.

На данную дату курс доллара США был 65,1 руб. Сегодня курс доллара 92,5 руб. Можем это учесть при отсечении выбросов и аномальных значений.

Создадим словарь, в который поместим наиболее реальные значения цен (проанализирован сайт вручную), которые уже на основе курса доллара записаны на момент '2019-10-10'.

In [15]:
max_price_dict = {'Автомобили': 105000000,
                  'Аквариум': 1050000,
                  'Аудио и видео': 1500000,
                  'Билеты и путешествия': 105000,
                  'Бытовая техника': 2110000,
                  'Вакансии': 850000,
                  'Велосипеды': 1500000,
                  'Водный транспорт': 350000000,
                  'Гаражи и машиноместа': 6300000,
                  'Готовый бизнес': 2500000000,
                  'Грузовики и спецтехника': 9150000,
                  'Детская одежда и обувь': 28000,
                  'Дома, дачи, коттеджи': 17600000,
                  'Другие животные': 180000,
                  'Запчасти и аксессуары': 615000,
                  'Земельные участки': 14000000000,
                  'Игры, приставки и программы': 20000,
                  'Квартиры': 1400000000,
                  'Книги и журналы': 2110000,
                  'Коллекционирование': 704000000,
                  'Коммерческая недвижимость': 350000000,
                  'Комнаты': 35200000,
                  'Кошки': 580000,
                  'Красота и здоровье': 14000,
                  'Мебель и интерьер': 800000,
                  'Мотоциклы и мототехника': 7000000,
                  'Музыкальные инструменты': 4200000,
                  'Настольные компьютеры': 550000,
                  'Недвижимость за рубежом': 1100000000,
                  'Ноутбуки': 900000,
                  'Оборудование для бизнеса': 110000000,
                  'Одежда, обувь, аксессуары': 400000,
                  'Оргтехника и расходники': 10000,
                  'Охота и рыбалка': 70000,
                  'Планшеты и электронные книги': 40000,
                  'Посуда и товары для кухни': 21120,
                  'Предложение услуг': 180000,
                  'Продукты питания': 70000,
                  'Птицы': 200000,
                  'Растения': 800000,
                  'Резюме': 350000,
                  'Ремонт и строительство': 704000,
                  'Собаки': 1100000000,
                  'Спорт и отдых': 100000,
                  'Телефоны': 1500000,
                  'Товары для детей и игрушки': 100000,
                  'Товары для животных': 600000,
                  'Товары для компьютера': 1000000,
                  'Фототехника': 5000000,
                  'Часы и украшения': 9000000}

In [16]:
date_get = pd.to_datetime('2019-10-10')

Теперь мы можем создать дополнительные признаки на основе даты выгрузки.

In [17]:
# функция для обработки пропусков
def fillna_median(dataset, column):
    filled_dataset = dataset.copy()
    filled_dataset[column] = filled_dataset.groupby('subcategory')[column].transform(lambda x: x.fillna(x.median()))
    return filled_dataset

In [18]:
# функция для создания токенов - упрощенная
def tokenizer(text):
    # все неалфавитные символы заменяются на пробелы
    text = re.sub(r'\W', ' ', text) 
    # символ '_' находится и удаляется.
    text = re.sub(r'_', '', text)
    # вставляет пробел перед нецифровым символом, который находится после цифры
    text = re.sub(r'(?<=\d)(\D)', r' \1', text)
    # вставляет пробел после нецифрового символа, который находится перед цифрой
    text = re.sub(r'(\D)(?=\d)', r'\1 ', text)
    # все цифры заменяются на 0.
    text = re.sub(r'\d', '0', text)
    return text.lower().split()

In [19]:
# функция для обнаружения номера телефона
def phone_num(text):
    if re.search(r'(\d.*){6,11}', text):
        return 1
    else:
        return 0
    
# функция для обнаружения номера телефона в формате РФ
def phone_rus(text):
    if re.search(r'(\+7|7|8)+([0-9]){10}', text):
        return 1
    else:
        return 0  

# функция для обнаружения номера телефона в описании без букв
def phone_no_letter(text):    
    if re.match(r'\d{6,11}', re.sub('\D', '', text)):
        return 1
    else:
        return 0

# функция для обнаружения номера телефона в описании без пробелов
def phone_no_space(text):
    if re.search(r'\d{6,11}', re.sub('\s', '', text)):
        return 1
    else:
        return 0

In [20]:
def data_processing(data, date_get, max_price_dict):
    # Переведем признак со временем в обрабатываемый тип данных
    data['datetime_submitted'] = pd.to_datetime(data['datetime_submitted'])
    data['days'] = date_get - data['datetime_submitted']
    
    # data_new['timedelta'] будет представлять временные интервалы в наносекундах в целочисленной форме
    data['timedelta'] = data['days'].dt.total_seconds().astype(int)
    
    # создадим новый признак - максимальная цена
    data['price_max'] = data['subcategory'].map(max_price_dict)
    
    # создадим новый признак - цена выше максимальной (аномалия)
    data['price_above_max'] = data['price'] > data['price_max']
    percentage_above_max = data.groupby('subcategory')['price_above_max'].mean() * 100
    print("Выбросы для каждой подкатегории:")
    print(percentage_above_max)
    
    # уберем явные выбросы из датасета на основе max_price_dict
    print("Для каждой группы это значение меньше 10% - удалим явные аномалии.")
    data_new = data[data['price_above_max'] == False]
    
    # Заполним пропущенные значения цены в датасете медианным значением подкатегории
    filled_dataset = fillna_median(data_new, 'price')
    
    # Значения цены выше 75го персентиля прированяем 75му персентилю
   # new_dataset = replace_outliers_with_quantile(filled_dataset, 'price')
    new_dataset = filled_dataset

    new_dataset['phone_num'] = new_dataset['description'].apply(phone_num)
    new_dataset['phone_rus'] = new_dataset['description'].apply(phone_rus)
    new_dataset['phone_no_letter'] = new_dataset['description'].apply(phone_no_letter)
    new_dataset['phone_no_space'] = new_dataset['description'].apply(phone_no_space)
    
    return new_dataset

In [21]:
data_processed = data_processing(data_new, date_get, max_price_dict)
print("="*50)
print("Новый датасет после предобработки")
data_processed.head()

Выбросы для каждой подкатегории:
subcategory
Автомобили                      0.008990
Аквариум                        0.000000
Аудио и видео                   0.037123
Билеты и путешествия            0.961538
Бытовая техника                 0.026526
Вакансии                        0.037839
Велосипеды                      0.000000
Водный транспорт                0.072780
Гаражи и машиноместа            0.170316
Готовый бизнес                  0.100301
Грузовики и спецтехника         1.077716
Детская одежда и обувь          0.034768
Дома, дачи, коттеджи            1.768308
Другие животные                 0.861020
Запчасти и аксессуары           0.033183
Земельные участки               0.033170
Игры, приставки и программы     6.404030
Квартиры                        0.015112
Книги и журналы                 0.028482
Коллекционирование              0.005350
Коммерческая недвижимость       0.851921
Комнаты                         0.043773
Кошки                           0.024528
Красота и зд

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,days,timedelta,price_max,price_above_max,phone_num,phone_rus,phone_no_letter,phone_no_space
0,Диван-кровать,Продаем диван-кровать. Удобный механизм - евро...,Мебель и интерьер,Для дома и дачи,7000.0,Россия,Москва,2019-06-01 00:00:15.180656,0,130 days 23:59:44.819344,11318384,800000,False,1,0,1,0
1,Кожух рулевой колонки Даф хф 91 4509834,Кожух рулевой колонки DAF XF 94 (60066004)/\n ...,Запчасти и аксессуары,Транспорт,2290.0,Россия,Москва,2019-06-01 00:00:44.317933,0,130 days 23:59:15.682067,11318355,615000,False,1,0,1,1
2,Дешёвый буст аккаунтов Dota 4,! Буст аккаунтов с ммр выше 1000ммр не беру ! ...,Предложение услуг,Услуги,200.0,Северная Осетия,Владикавказ,2019-06-01 00:00:50.249692,1,130 days 23:59:09.750308,11318349,180000,False,1,0,1,1
3,Телевизор sharp.Смарт тв.Интернет,Продам телевизор . Диагональ 450.наличие входа...,Аудио и видео,Бытовая электроника,25000.0,Калининградская область,Советск,2019-06-01 00:00:50.325799,1,130 days 23:59:09.674201,11318349,1500000,False,0,0,0,0
4,Открытка-конверт,Открытки-конверты ручной работы/\nВыполнены в ...,Коллекционирование,Хобби и отдых,150.0,Ставропольский край,Ессентукская,2019-06-01 00:00:56.632655,0,130 days 23:59:03.367345,11318343,704000000,False,0,0,0,0


## Валидационная выборка

In [22]:
data_valid, data_valid_missed, data_valid_info, data_valid_statistics = initial_dataset_analysis(valid_path)

print("\nData Information:")
info_lines = data_valid_info.splitlines()
for line in info_lines:
    print(line)
print("=" * 10)

print("\nData missed values:")
display(data_valid_missed)
print("=" * 10)

print("Summary Statistics:")
display(data_valid_statistics)
print("=" * 10)

print("\nData:")
print(data_valid.head())


Data Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16237 entries, 0 to 16236
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               16237 non-null  object 
 1   description         16237 non-null  object 
 2   subcategory         16237 non-null  object 
 3   category            16237 non-null  object 
 4   price               15374 non-null  float64
 5   region              16237 non-null  object 
 6   city                16237 non-null  object 
 7   datetime_submitted  16237 non-null  object 
 8   is_bad              16237 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 1.1+ MB

Data missed values:


,0
title,0.000000
description,0.000000
subcategory,0.000000
category,0.000000
price,5.300000
region,0.000000
city,0.000000
datetime_submitted,0.000000
is_bad,0.000000


Summary Statistics:


,price,is_bad
count,1.537400e+04,16237.000000
mean,6.986385e+06,0.245181
std,7.247229e+08,0.430208
min,0.000000e+00,0.000000
25%,1.500000e+03,0.000000
50%,6.500000e+03,0.000000
75%,4.500000e+04,0.000000
max,8.929893e+10,1.000000



Data:
                                      title  \
0                                      Шины   
1  Продается мобильная перегородка с дверью   
2   Комплект зимних шин на дисках 682/32/64   
3         Кровать-трансормер Дакота сб-4085   
4                    Honda VFR 800 2004 г.в   

                                         description  \
0                                Звонить 89425546881   
1  Мобильная перегородка, предназначена для разгр...   
2  Шины зимние б/у Marshal Assimetric I”Zen KW 61...   
3  Продаю кровать-трансформер производства "Столп...   
4  Мот в отличном состоянии для своих лет, Родной...   

                subcategory         category     price               region  \
0     Запчасти и аксессуары        Транспорт    2000.0     Тульская область   
1  Оборудование для бизнеса      Для бизнеса   10500.0  Вологодская область   
2     Запчасти и аксессуары        Транспорт    4000.0               Россия   
3         Мебель и интерьер  Для дома и дачи   17000.0   

Произведем аналогичную предобработку для валидационного датасета.

In [23]:
val = data_valid.copy()

In [24]:
val_processed = data_processing(val, date_get, max_price_dict)
print("="*50)
print("Новый датасет после предобработки")
val_processed.head()

Выбросы для каждой подкатегории:
subcategory
Автомобили                       0.183318
Аквариум                         0.000000
Аудио и видео                    0.000000
Билеты и путешествия             4.166667
Бытовая техника                  0.000000
Вакансии                         0.000000
Велосипеды                       0.000000
Водный транспорт                 0.000000
Гаражи и машиноместа             0.000000
Готовый бизнес                   0.000000
Грузовики и спецтехника          0.643087
Детская одежда и обувь           0.000000
Дома, дачи, коттеджи             1.879699
Другие животные                  0.000000
Запчасти и аксессуары            0.049863
Земельные участки                0.000000
Игры, приставки и программы      4.545455
Квартиры                         0.000000
Книги и журналы                  0.000000
Коллекционирование               0.000000
Коммерческая недвижимость        0.952381
Комнаты                          0.000000
Кошки                          

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,days,timedelta,price_max,price_above_max,phone_num,phone_rus,phone_no_letter,phone_no_space
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1,-1 days +23:59:34.799286,-25,615000,False,1,1,1,1
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0,-1 days +23:56:48.472708,-191,110000000,False,1,0,1,0
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1,-1 days +23:54:52.806752,-307,615000,False,1,0,1,1
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0,-1 days +23:54:01.834821,-358,800000,False,0,0,1,0
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0,-1 days +23:53:40.768849,-379,7000000,False,1,0,1,0


In [25]:
# Класс, который выбранные признаки оставляет без изменений
class Not_changed(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        self.columns = X.columns
        return self
  
    def transform(self, X, y=None):
        return X

    def get_feature_names(self):
        return self.columns  

Создание пайплайна модели:

In [26]:
%%time

pipeline = make_column_transformer(
    #(CountVectorizer(min_df=1000), 'city'),
    (CountVectorizer(min_df=1000, token_pattern=r'.+'), 'category'),
    (Not_changed(), ['price', 'timedelta', 'phone_num', 'phone_rus', 'phone_no_letter', 'phone_no_space']),
    (CountVectorizer(min_df=1000,
                     analyzer='word',
                     ngram_range=(1,3),
                     tokenizer=tokenizer), 'description')
    )

CPU times: user 79 µs, sys: 1 µs, total: 80 µs
Wall time: 84.2 µs


## Подготовка данных к обучению

Выделим признаки, а также целевой признак для обучения.

In [27]:
target_train = data_processed['is_bad'] # целевой признак
features_train = data_processed.drop(['is_bad', 'price_above_max', 
                                      'price_max', 'price_above_max', 
                                      'days', 'datetime_submitted'], axis=1) # признаки

In [28]:
target_valid = val_processed['is_bad'] # целевой признак
features_valid = val_processed.drop(['is_bad', 'price_above_max', 
                                     'price_max', 'price_above_max', 
                                     'days', 'datetime_submitted'], axis=1) # признаки

In [29]:
%%time
X_train = pipeline.fit_transform(features_train)
X_valid = pipeline.transform(features_valid)
  
Y_train = data_processed.is_bad
Y_valid = val_processed.is_bad

/opt/conda/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CPU times: user 8min 11s, sys: 10 s, total: 8min 21s
Wall time: 8min 21s


In [30]:
X_train.shape

(980916, 14893)

Так как у нас разреженная матрица, то мы не можем ее просто как признаки подать в CatBoostClassifier. Нужно подготовить специальный объект.

In [31]:
from catboost import CatBoostClassifier, Pool
# Создание Pool объекта, в который передаются данные и их имена признаков
train_pool = Pool(data=X_train, label=Y_train)

In [32]:
grid_cat = CatBoostClassifier(iterations=1000,
                              depth=12,
                              learning_rate=0.03,
                              l2_leaf_reg=3,
                              min_data_in_leaf=30,
                              border_count=64,
                              bagging_temperature=0.5,
                              eval_metric='AUC',
                              use_best_model=True,
                              random_state=RANDOM_STATE, 
                              auto_class_weights='Balanced',
                              task_type='GPU',
                              silent=True)


grid_cat.fit(train_pool,
             verbose=50,
             eval_set=(X_valid, Y_valid),
             early_stopping_rounds=30)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8656811	best: 0.8656811 (0)	total: 3.72s	remaining: 1h 1m 52s
50:	test: 0.9313095	best: 0.9313095 (50)	total: 28.2s	remaining: 8m 44s
100:	test: 0.9457046	best: 0.9457046 (100)	total: 51s	remaining: 7m 34s
150:	test: 0.9516083	best: 0.9516083 (150)	total: 1m 13s	remaining: 6m 51s
200:	test: 0.9547615	best: 0.9547615 (200)	total: 1m 34s	remaining: 6m 17s
250:	test: 0.9568910	best: 0.9568975 (249)	total: 1m 56s	remaining: 5m 47s
300:	test: 0.9586331	best: 0.9586331 (300)	total: 2m 18s	remaining: 5m 21s
350:	test: 0.9601145	best: 0.9601145 (350)	total: 2m 40s	remaining: 4m 56s
400:	test: 0.9611386	best: 0.9611386 (400)	total: 3m 1s	remaining: 4m 31s
450:	test: 0.9618896	best: 0.9618897 (449)	total: 3m 23s	remaining: 4m 7s
500:	test: 0.9626857	best: 0.9626857 (500)	total: 3m 45s	remaining: 3m 44s
550:	test: 0.9634533	best: 0.9634628 (549)	total: 4m 7s	remaining: 3m 21s
600:	test: 0.9640241	best: 0.9640241 (600)	total: 4m 29s	remaining: 2m 59s
650:	test: 0.9647296	best: 0.9647296

# Сохраняем веса модели

In [ ]:
grid_cat.save_model('/kaggle/working/model_cat.cbm')

In [33]:
ra_cat = roc_auc_score(Y_valid, grid_cat.predict_proba(X_valid)[:, 1])

Посчитаем усредненный ROC-AUC по каждой категории объявлений.

In [34]:
categories = val_processed['category'].unique()
mean_roc_auc = 0.0

# Проходим по каждой категории
for category in categories:
    # Выбираем только строки с текущей категорией
    category_data = val_processed[val_processed['category'] == category]
    
    category_features = pipeline.transform(category_data.drop(['is_bad', 
                                                               'price_above_max', 
                                                               'price_max', 
                                                               'price_above_max', 
                                                               'days', 
                                                               'datetime_submitted'], 
                                                              axis=1))
    
    # ROC-AUC по каждой категории
    roc_auc = roc_auc_score(category_data['is_bad'], 
                            grid_cat.predict_proba(category_features)[:, 1])
    
    # Печать ROC-AUC для каждой категории (опционально)
    print(f'ROC-AUC for {category}: {roc_auc}')
    
    # Добавление к текущему среднему
    mean_roc_auc += roc_auc

# Усреднение ROC-AUC по всем категориям
mean_roc_auc /= len(categories)

# Печать усредненного ROC-AUC
print(f'Mean ROC-AUC: {mean_roc_auc}')

ROC-AUC for Транспорт: 0.9897136969368205
ROC-AUC for Для бизнеса: 0.8559986923831318
ROC-AUC for Для дома и дачи: 0.9500775487043692
ROC-AUC for Личные вещи: 0.804363468699133
ROC-AUC for Услуги: 0.8981751168644372
ROC-AUC for Бытовая электроника: 0.9427179091789732
ROC-AUC for Недвижимость: 0.9822510358248933
ROC-AUC for Хобби и отдых: 0.9522477522477523
ROC-AUC for Работа: 0.9158802783338099
ROC-AUC for Животные: 0.8955102040816326
Mean ROC-AUC: 0.9186935703254953


# Вывод

Была создана модель для проверки наличия контактной информации в тексте объявления. 

В данной задаче использовался `CatBoostClassifier`, который при первом приближении дал результат `0,72`. Дополнительные признаки, улучшили значение метрики до `0,918`. 

Также есть идея, что можно получить хорошие результаты на основе обучения Q/A модели, которая сможет предсказать начало и конец контактной информации после дообучения.